In [ ]:
!pip install bambi
!pip install arviz
!pip install snowflake.connector

In [38]:
import snowflake.connector
import pandas as pd
import numpy as np
import bambi as bmb
import arviz as az


WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def data_required(conn_details,table_details,filter_details):
    import snowflake.connector
    conn = snowflake.connector.connect(
    user = conn_details['user'],
    password = conn_details['password'],
    role = conn_details['role'],
    account = conn_details['account'],
    warehouse = conn_details['warehouse'],
    database = table_details['database'],
    schema = table_details['schema'])
    cur=conn.cursor()
    filter_condition = f""
    for key in filter_details.keys():
        filter_condition+= key+' = ' + str(filter_details[key]) + ' and '
    filter_condition = filter_condition[:-5]
    query = f"SELECT * FROM  {table_details['tablename']} WHERE "+filter_condition
    print(query)
    cur.execute(query)
    dat = pd.DataFrame.from_records(iter(cur),columns=[x[0] for x in cur.description])
    cur.close()
    return dat
    

In [3]:
conn_details ={
    "user": "RBHASKAR",
    "password": "Extropy360@1",
    "role": 'DATABASE_DEV',
    'account': "FHA02592.us-east-1",
    'warehouse': "COMPUTE_WH"
}
table_details = {
    "database": "ARB_PRIMANTI",
    "schema": "ETL",
    "tablename": '"VW_PRODUCT_MIX"'
}

table_details_1  = {
    "database": "ARB_PRIMANTI",
    "schema": "ETL",
    "tablename": '"VW_TRANSACTION_DAILY"'
}


## Basket Elasticity

In [123]:
Final_Results = pd.DataFrame()
Secondary_data = pd.DataFrame()
Primary_data = pd.DataFrame()
for client in [1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,29,30,31,33,36,37,38,39,40,41,42,43,44,45,46]:
    filter_details = {
    "CLIENT_LOCATION_ID": client,
    "REVENUE_CENTER": "'Restaurant'",}
    processed_data = data_required(conn_details,table_details,filter_details)
    normalised_data = data_required(conn_details,table_details_1,filter_details)
    
    filter_data = processed_data[['PERIOD', 'CLIENT_LOCATION_ID', 'MEAL_PERIOD', 'REVENUE_CENTER','QUANTITY', 'SALES_GROSS', 'SALES_NET', 'SALES_DISCOUNT', 'CLIENT_ID','PRODUCT_NAME', 'CATEGORY_1', 'CATEGORY_2_E360','CATEGORY_3_E360', 'CATEGORY_4_E360', 'CATEGORY_5_E360','MENU_CATEGORY']]
    filter_data['year'] = pd.to_datetime(filter_data['PERIOD']).dt.year
    filter_data['price_per_item'] = filter_data['SALES_NET']/filter_data['QUANTITY']
    filter_data = pd.merge(filter_data,normalised_data[['PERIOD','MEAL_PERIOD','TRANS_COUNT']],on=['PERIOD','MEAL_PERIOD'],how='left')
    filter_data['QUANTITY_NORMALISED'] = filter_data['QUANTITY']/filter_data['TRANS_COUNT']
    # filter_data['QUANTITY_NORMALISED'] = filter_data['QUANTITY']
    filter_data['WEEK'] = pd.to_datetime(filter_data['PERIOD']).dt.to_period('W')
    filter_data['WEEK'] = filter_data['WEEK'].dt.start_time
    filter_data = filter_data[filter_data['SALES_NET']>0]
    rolled_data = filter_data.groupby(['WEEK','CLIENT_LOCATION_ID','PRODUCT_NAME','CATEGORY_1', 'CATEGORY_2_E360','CATEGORY_3_E360', 'CATEGORY_4_E360', 'CATEGORY_5_E360','MENU_CATEGORY']).agg(QUANTITY = ("QUANTITY_NORMALISED",'sum'),PRICE_PER_ITEM = ('price_per_item','mean')).reset_index()
    product_mapping = pd.read_csv("C:\pricing\Primanti_product_mapping.csv")
    product_mapping.rename(columns={"Product Name":"PRODUCT_NAME"},inplace=True)
    rolled_data = pd.merge(rolled_data,product_mapping[['PRODUCT_NAME','analysis','major_category','minor_category']],on='PRODUCT_NAME')
    rolled_data = rolled_data[rolled_data['analysis']=='Considered']
    secondary_data = rolled_data.query("major_category=='secondary_item'").groupby(['WEEK','major_category','minor_category']).agg(QUANTITY = ('QUANTITY','sum')).reset_index()
    secondary_data['CLIENT_LOCATION_ID'] = rolled_data['CLIENT_LOCATION_ID'].unique()[0]
    top_primary_items = rolled_data.query('WEEK>="2022-03-08" and major_category =="primary_item"').groupby(['PRODUCT_NAME']).agg(QUANTITY = ('QUANTITY','sum')).reset_index().sort_values(by='QUANTITY',ascending=False)[:5]
    top_primary_items= top_primary_items[top_primary_items['PRODUCT_NAME']!='#NAME?']
    primary_price_data = rolled_data.loc[rolled_data['PRODUCT_NAME'].isin(top_primary_items['PRODUCT_NAME'].to_list()),['WEEK','PRODUCT_NAME','PRICE_PER_ITEM']]
    primary_price_data = pd.pivot_table(primary_price_data,index=['WEEK'],columns='PRODUCT_NAME').reset_index()
    primary_price_data.columns = primary_price_data.columns.droplevel(0)
    primary_price_data = primary_price_data.reset_index().rename_axis(None, axis=1)
    primary_price_data['CLIENT_LOCATION_ID'] = rolled_data['CLIENT_LOCATION_ID'].unique()[0]
    primary_price_data.rename(columns={"":"WEEK"},inplace=True)
    primary_price_data.drop(columns=['index'],inplace=True)
    model_data = pd.merge(secondary_data,primary_price_data,how='left',on=['WEEK',"CLIENT_LOCATION_ID"])
    model_data.dropna(inplace=True)
    product_data = model_data.groupby(['minor_category'])
    for p,g in product_data:
        prod_data = pd.DataFrame(g)    
        prod_data.columns = [col.replace(" ","_") for col in prod_data.columns]
        prod_data.columns = [col.replace("-","_") for col in prod_data.columns]
        prod_data['QUANTITY'] = np.log((1+prod_data['QUANTITY'])*1000)
        req_cols = [col for col in prod_data.columns if col not in ['WEEK','major_category','minor_category','QUANTITY','CLIENT_LOCATION_ID']]
        formula1 = 'QUANTITY ~ '
        for column in req_cols:
            prod_data[column] = np.log(1+prod_data[column])
            formula1 = formula1 +column +" + "
        formula1 = formula1[:-3]
        model = bmb.Model(formula=formula1,data=prod_data,family="gaussian")
        results = az.summary(model.fit())
        results['minor_category'] = p
        results['CLIENT_LOCATION_ID'] = rolled_data['CLIENT_LOCATION_ID'].unique()[0]
        Final_Results = pd.concat([Final_Results,results])


SELECT * FROM  "VW_PRODUCT_MIX" WHERE CLIENT_LOCATION_ID = 1 and REVENUE_CENTER = 'Restaurant'
SELECT * FROM  "VW_TRANSACTION_DAILY" WHERE CLIENT_LOCATION_ID = 1 and REVENUE_CENTER = 'Restaurant'


C:\Users\kbhas\AppData\Local\Temp\ipykernel_16520\4022519982.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data['year'] = pd.to_datetime(filter_data['PERIOD']).dt.year
C:\Users\kbhas\AppData\Local\Temp\ipykernel_16520\4022519982.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data['price_per_item'] = filter_data['SALES_NET']/filter_data['QUANTITY']


## Category Elasticity

In [ ]:
Final_Results = pd.DataFrame()
for client in [1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,29,30,31,33,36,37,38,39,40,41,42,43,44,45,46]:
    filter_details = {
    "CLIENT_LOCATION_ID": client,
    "REVENUE_CENTER": "'Restaurant'",}
    processed_data = data_required(conn_details,table_details,filter_details)
    normalised_data = data_required(conn_details,table_details_1,filter_details)
    filter_data = processed_data[['PERIOD', 'CLIENT_LOCATION_ID', 'MEAL_PERIOD', 'REVENUE_CENTER','QUANTITY', 'SALES_GROSS', 'SALES_NET', 'SALES_DISCOUNT', 'CLIENT_ID','PRODUCT_NAME', 'CATEGORY_1', 'CATEGORY_2_E360','CATEGORY_3_E360', 'CATEGORY_4_E360', 'CATEGORY_5_E360','MENU_CATEGORY']]
    filter_data['year'] = pd.to_datetime(filter_data['PERIOD']).dt.year
    filter_data['price_per_item'] = filter_data['SALES_NET']/filter_data['QUANTITY']
    filter_data = pd.merge(filter_data,normalised_data[['PERIOD','MEAL_PERIOD','TRANS_COUNT']],on=['PERIOD','MEAL_PERIOD'],how='left')
    # filter_data['QUANTITY_NORMALISED'] = filter_data['QUANTITY']/filter_data['TRANS_COUNT']
    filter_data['QUANTITY_NORMALISED'] = filter_data['QUANTITY']
    filter_data['WEEK'] = pd.to_datetime(filter_data['PERIOD']).dt.to_period('W')
    filter_data['WEEK'] = filter_data['WEEK'].dt.start_time
    filter_data = filter_data[filter_data['SALES_NET']>0]
    rolled_data = filter_data.groupby(['WEEK','CLIENT_LOCATION_ID','PRODUCT_NAME','CATEGORY_1', 'CATEGORY_2_E360','CATEGORY_3_E360', 'CATEGORY_4_E360', 'CATEGORY_5_E360','MENU_CATEGORY']).agg(QUANTITY = ("QUANTITY_NORMALISED",'sum'),PRICE_PER_ITEM = ('price_per_item','mean')).reset_index()
    product_mapping = pd.read_csv("C:\pricing\Primanti_product_mapping.csv")
    product_mapping.rename(columns={"Product Name":"PRODUCT_NAME"},inplace=True)
    rolled_data = pd.merge(rolled_data,product_mapping[['PRODUCT_NAME','analysis','major_category','minor_category']],on='PRODUCT_NAME')
    rolled_data = rolled_data[rolled_data['analysis']=='Considered']
    secondary_data = rolled_data.query("major_category=='secondary_item'").groupby(['WEEK','major_category','minor_category']).agg(QUANTITY = ('QUANTITY','sum')).reset_index()
    primary_data = rolled_data.query("major_category=='primary_item'").groupby(['WEEK','major_category','minor_category']).agg(PRICE_PER_ITEM = ('PRICE_PER_ITEM','mean')).reset_index()
    primary_price_data = pd.pivot_table(primary_data,index=['WEEK'],columns='minor_category').reset_index()
    primary_price_data.columns = primary_price_data.columns.droplevel(0)
    primary_price_data = primary_price_data.reset_index().rename_axis(None, axis=1)
    primary_price_data.rename(columns={"":"WEEK"},inplace=True)
    primary_price_data.drop(columns=['index'],inplace=True)
    model_data = pd.merge(secondary_data,primary_price_data,how='left',on='WEEK')
    model_data.dropna(inplace=True)
    product_data = model_data.groupby(['minor_category'])
    for p,g in product_data:
        prod_data = pd.DataFrame(g)
        prod_data['QUANTITY'] = np.log((1+prod_data['QUANTITY'])*1000)        
        for column in prod_data.columns:
          if column in ['pizza','salad','sandwiches']:
            prod_data[column] = np.log(1+prod_data[column])
            formula12 = 'QUANTITY ~ '+column
            print(formula12)
            model = bmb.Model(formula=formula12,data=prod_data,family="gaussian")
            results = az.summary(model.fit())
            results['minor_category'] = p
            results['CLIENT_LOCATION_ID'] = rolled_data['CLIENT_LOCATION_ID'].unique()[0]
            del formula12
            Final_Results = pd.concat([Final_Results,results])
          
Final_Results.to_csv('/content/Major_Minor_elasticty_Results.csv')